In [18]:
# Import Libraries
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from ta.others import daily_return,cumulative_return
from plotly.offline import init_notebook_mode
import cufflinks as cf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import requests
from bs4 import BeautifulSoup
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline()

In [19]:
# Check if folder input exists

# Define the folder name
folder_name = '../input'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../input' folder already exists.


In [20]:
# Check if folder tickers exists

# Define the folder name
folder_name = '../tickers'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../tickers' folder already exists.


In [21]:
# Webscrapping Wikipedia table to get the DAX40 components
page = requests.get("https://en.wikipedia.org/wiki/MDAX")
soup = BeautifulSoup(page.text, 'html.parser') 
table = soup.find('table',id='constituents',class_="wikitable sortable zebra")

In [22]:
# Convert html table to dataframe
wiki = pd.read_html(str(table))
wiki = pd.concat(wiki)
wiki.head(40)

Logo                       Name                                Industry  \
0    NaN                 Aixtron SE                  Semiconductor industry   
1    NaN            Aroundtown S.A.                             Real estate   
2    NaN                 Aurubis AG                                  Metals   
3    NaN                 Bechtle AG                             IT services   
4    NaN                Befesa S.A.                        Waste management   
5    NaN               Bilfinger SE                                     NaN   
6    NaN      Carl Zeiss Meditec AG                      Medical technology   
7    NaN  CTS Eventim AG & Co. KGaA                          Leisure-events   
8    NaN              Delivery Hero                        Delivery service   
9    NaN      Deutsche Lufthansa AG                                Airlines   
10   NaN                 Encavis AG                        Renewable energy   
11   NaN       Evonik Industries AG                               Chemistry   
12   NaN                  Evotec SE                           Biotechnology   
13   NaN                 Fraport AG                        Airport operator   
14   NaN                 Freenet AG                      Telecommunications   
15   NaN  Fresenius Medical Care AG                      Medical technology   
16   NaN          Fuchs Petrolub SE                               Chemistry   
17   NaN               GEA Group AG                               Machinery   
18   NaN            Gerresheimer AG                               Packaging   
19   NaN      HELLA GmbH & Co. KGaA                              Automotive   
20   NaN              HelloFresh SE                                Meal kit   
21   NaN                Hensoldt AG                                     NaN   
22   NaN                Hochtief AG                            Construction   
23   NaN               Hugo Boss AG                   Clothing, Accessories   
24   NaN                Jenoptik AG                                     NaN   
25   NaN            Jungheinrich AG  Intralogistics, Mechanical engineering   
26   NaN                     K+S AG            Mining (fertilizer and salt)   
27   NaN              Kion Group AG                      Handling equipment   
28   NaN            Knorr-Bremse AG                           Manufacturing   
29   NaN                     Krones                                     NaN   
30   NaN                 Lanxess AG                               Chemistry   
31   NaN          LEG Immobilien AG                             Real estate   
32   NaN              Nemetschek SE                                Software   
33   NaN                  Nordex SE                                     NaN   
34   NaN                    PUMA SE                        Sports equipment   
35   NaN                Rational AG                           Manufacturing   
36   NaN      Redcare Pharmacy N.V.                                Pharmacy   
37   NaN               RTL Group SA                                   Media   
38   NaN                 Scout24 AG                           Online-market   
39   NaN               Siltronic AG                  Semiconductor industry   

    Index weighting Number of Shares[6] Free-float-  in Mio. €  \
0            298.00        0113.402.370               4.325,62   
1              1.49          1536397797                2061.72   
2            159.00        0044.956.723               2.307,09   
3            257.00        0126.000.000               3.741,68   
4              1.11                 NaN                1530.47   
5               NaN                 NaN                    NaN   
6            245.00        0089.440.570               3.555,14   
7            256.00        0096.000.000               3.713,64   
8            423.00        0269.536.421               6.136,97   
9            569.00       1.195.485.644               8.267,54   
10           123.00        0161.030.17

In [23]:
wiki['Symbol'] = wiki['Symbol'].apply(lambda x: x + '.DE')
wiki

Logo                       Name                                Industry  \
0    NaN                 Aixtron SE                  Semiconductor industry   
1    NaN            Aroundtown S.A.                             Real estate   
2    NaN                 Aurubis AG                                  Metals   
3    NaN                 Bechtle AG                             IT services   
4    NaN                Befesa S.A.                        Waste management   
5    NaN               Bilfinger SE                                     NaN   
6    NaN      Carl Zeiss Meditec AG                      Medical technology   
7    NaN  CTS Eventim AG & Co. KGaA                          Leisure-events   
8    NaN              Delivery Hero                        Delivery service   
9    NaN      Deutsche Lufthansa AG                                Airlines   
10   NaN                 Encavis AG                        Renewable energy   
11   NaN       Evonik Industries AG                               Chemistry   
12   NaN                  Evotec SE                           Biotechnology   
13   NaN                 Fraport AG                        Airport operator   
14   NaN                 Freenet AG                      Telecommunications   
15   NaN  Fresenius Medical Care AG                      Medical technology   
16   NaN          Fuchs Petrolub SE                               Chemistry   
17   NaN               GEA Group AG                               Machinery   
18   NaN            Gerresheimer AG                               Packaging   
19   NaN      HELLA GmbH & Co. KGaA                              Automotive   
20   NaN              HelloFresh SE                                Meal kit   
21   NaN                Hensoldt AG                                     NaN   
22   NaN                Hochtief AG                            Construction   
23   NaN               Hugo Boss AG                   Clothing, Accessories   
24   NaN                Jenoptik AG                                     NaN   
25   NaN            Jungheinrich AG  Intralogistics, Mechanical engineering   
26   NaN                     K+S AG            Mining (fertilizer and salt)   
27   NaN              Kion Group AG                      Handling equipment   
28   NaN            Knorr-Bremse AG                           Manufacturing   
29   NaN                     Krones                                     NaN   
30   NaN                 Lanxess AG                               Chemistry   
31   NaN          LEG Immobilien AG                             Real estate   
32   NaN              Nemetschek SE                                Software   
33   NaN                  Nordex SE                                     NaN   
34   NaN                    PUMA SE                        Sports equipment   
35   NaN                Rational AG                           Manufacturing   
36   NaN      Redcare Pharmacy N.V.                                Pharmacy   
37   NaN               RTL Group SA                                   Media   
38   NaN                 Scout24 AG                           Online-market   
39   NaN               Siltronic AG                  Semiconductor industry   
40   NaN                STABILUS SE                              Automotive   
41   NaN       Ströer SE & Co. KGaA                                   Media   
42   NaN          TAG Immobilien AG                             Real estate   
43   NaN                  Talanx AG                             Real Estate   
44   NaN              TeamViewer AG                                Software   
45   NaN            Thyssenkrupp AG                            Conglomerate   
46   NaN                  Traton SE                                     NaN   
47   NaN                     TUI AG                                 Tourism   
48   NaN         United Internet AG                      Telecommunications   
49   NaN           Wacker Chemie AG                   

In [24]:
wiki.rename(columns={'Industry':'Sector','Symbol':'Ticker'}, inplace=True)

In [25]:
# Add index row values
new_row = {'Name': 'DAXMidcap', 'Sector': 'Index', 'Ticker': '^MDAXI'}

# Convert the new row to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Append the new row to the DataFrame
wiki = pd.concat([wiki[['Name','Sector','Ticker']], new_row_df], ignore_index=True)
wiki.head(41)

Name                                  Sector   Ticker
0                  Aixtron SE                  Semiconductor industry  AIXA.DE
1             Aroundtown S.A.                             Real estate   AT1.DE
2                  Aurubis AG                                  Metals   NDA.DE
3                  Bechtle AG                             IT services   BC8.DE
4                 Befesa S.A.                        Waste management  BFSA.DE
5                Bilfinger SE                                     NaN   GBF.DE
6       Carl Zeiss Meditec AG                      Medical technology   AFX.DE
7   CTS Eventim AG & Co. KGaA                          Leisure-events   EVD.DE
8               Delivery Hero                        Delivery service  DHER.DE
9       Deutsche Lufthansa AG                                Airlines   LHA.DE
10                 Encavis AG                        Renewable energy   ECV.DE
11       Evonik Industries AG                               Chemistry   EVK.DE
12                  Evotec SE                           Biotechnology   EVT.DE
13                 Fraport AG                        Airport operator   FRA.DE
14                 Freenet AG                      Telecommunications  FNTN.DE
15  Fresenius Medical Care AG                      Medical technology   FRE.DE
16          Fuchs Petrolub SE                               Chemistry  FPE3.DE
17               GEA Group AG                               Machinery   G1A.DE
18            Gerresheimer AG                               Packaging   GXI.DE
19      HELLA GmbH & Co. KGaA                              Automotive   HLE.DE
20              HelloFresh SE                                Meal kit   HFG.DE
21                Hensoldt AG                                     NaN   HAG.DE
22                Hochtief AG                            Construction   HOT.DE
23               Hugo Boss AG                   Clothing, Accessories  BOSS.DE
24                Jenoptik AG                                     NaN   JEN.DE
25            Jungheinrich AG  Intralogistics, Mechanical engineering  JUN3.DE
26                     K+S AG            Mining (fertilizer and salt)   SDF.DE
27              Kion Group AG                      Handling equipment   KGX.DE
28            Knorr-Bremse AG                           Manufacturing   KBX.DE
29                     Krones                                     NaN   KRN.DE
30                 Lanxess AG                               Chemistry   LXS.DE
31          LEG Immobilien AG                             Real estate   LEG.DE
32              Nemetschek SE                                Software   NEM.DE
33                  Nordex SE                                     NaN  NDX1.DE
34                    PUMA SE                        Sports equipment   PUM.DE
35                Rational AG                           Manufacturing   RAA.DE
36      Redcare Pharmacy N.V.                                Pharmacy   RDC.DE
37               RTL Group SA                                   Media  RRTL.DE
38                 Scout24 AG                           Online-market   G24.DE
39               Siltronic AG                  Semiconductor industry   WAF.DE
40                STABILUS SE                              Automotive   STM.DE

In [26]:
# Get the data for the stock index
index_list = wiki['Ticker'].tolist()

In [27]:
# Save all the historical data
for stock in tqdm(index_list):
    data = yf.download(stock, progress=False,multi_level_index=False,actions=True,auto_adjust=False)
    data.to_csv(f"../input/{stock}.csv",index=True)

  0%|          | 0/51 [00:00<?, ?it/s]

In [28]:
# Get the name of the notebook
notebook_name = os.path.basename(globals()['__vsc_ipynb_file__'])
notebook_name = notebook_name.split('-')[0]

In [29]:
# Save all the tickers data
wiki[['Name','Sector','Ticker']].to_csv(f"../tickers/{notebook_name}.csv",index=True)

In [30]:
# Configuration of different parameters of the notebook
ticker = '^MDAXI'
year = '2025'

In [31]:
# Check DataFrame
stock_ticker = pd.read_csv(f"../input/{ticker}.csv",index_col="Date",parse_dates=True)
stock_ticker.head(10)

Adj Close        Close  Dividends         High          Low  \
Date                                                                        
1996-02-29  2622.889893  2622.889893        0.0  2622.889893  2622.889893   
1996-03-01  2643.760010  2643.760010        0.0  2643.760010  2643.760010   
1996-03-04  2641.389893  2641.389893        0.0  2641.389893  2641.389893   
1996-03-05  2638.810059  2638.810059        0.0  2638.810059  2638.810059   
1996-03-06  2619.360107  2619.360107        0.0  2619.360107  2619.360107   
1996-03-07  2598.090088  2598.090088        0.0  2598.090088  2598.090088   
1996-03-08  2572.709961  2572.709961        0.0  2572.709961  2572.709961   
1996-03-11  2541.600098  2541.600098        0.0  2541.600098  2541.600098   
1996-03-12  2551.899902  2551.899902        0.0  2551.899902  2551.899902   
1996-03-13  2544.489990  2544.489990        0.0  2544.489990  2544.489990   

                   Open  Stock Splits  Volume  
Date                                           
1996-02-29  2622.889893           0.0       0  
1996-03-01  2643.760010           0.0       0  
1996-03-04  2641.389893           0.0       0  
1996-03-05  2638.810059           0.0       0  
1996-03-06  2619.360107           0.0       0  
1996-03-07  2598.090088           0.0       0  
1996-03-08  2572.709961           0.0       0  
1996-03-11  2541.600098           0.0       0  
1996-03-12  2551.899902           0.0       0  
1996-03-13  2544.489990           0.0       0

In [32]:
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':3,'colspan':1}],[None],[None],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Adj Close'],mode="lines",name=f"{ticker}"),row=1, col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Volume'],mode="lines",name='Volume'),row=4, col=1)
fig.update_yaxes(title_text="Volume", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker}")

fig.show()

In [33]:
# Group per year and calculate cummulative return
stock_ticker['Year'] = stock_ticker.index.year
annual_data = stock_ticker.groupby('Year').agg(Adj_Close=('Adj Close','last'))
annual_data['Return'] = annual_data["Adj_Close"].pct_change()*100
annual_data

Adj_Close     Return
Year                         
1996   2961.659912        NaN
1997   3684.179932  24.395779
1998   3905.449951   6.005950
1999   4103.819824   5.079309
2000   4675.339844  13.926538
2001   4326.120117  -7.469398
2002   3024.820068 -30.080072
2003   4469.229980  47.751928
2004   5375.740234  20.283366
2005   7311.529785  36.009730
2006   9404.889648  28.630942
2007   9864.620117   4.888207
2008   5601.910156 -43.212105
2009   7507.040039  34.008576
2010  10128.120117  34.914961
2011   8897.809570 -12.147472
2012  11914.370117  33.902283
2013  16574.449219  39.113097
2014  16934.849609   2.174434
2015  20774.619141  22.673774
2016  22188.939453   6.807924
2017  26200.769531  18.080315
2018  21588.089844 -17.605131
2019  28312.800781  31.150097
2020  30796.259766   8.771506
2021  35123.250000  14.050376
2022  25117.570312 -28.487340
2023  27137.300781   8.041106
2024  25589.060547  -5.705211
2025  30289.609375  18.369369

In [34]:
# Plotly
fig = make_subplots()

# Add colors
colors = ['green' if x >= 0 else 'red' for x in annual_data['Return']]

# Graph (1,1)
fig.add_trace(go.Bar(y=annual_data['Return'],x=annual_data['Return'].index,text=round(annual_data['Return'],2),textposition='outside',marker_color=colors,hoverinfo='skip'),row=1,col=1)

# Update xaxis properties
fig.update_yaxes(title_text="Returns", row=1, col=1)

fig.update_layout(xaxis=dict(tickvals=annual_data.index,tickangle=-45),height=800, width=1300,title=f"Returns per year {ticker}")

fig.show()

In [35]:
# Add Technical Analysis Indicators

# Modified Moving Average 20
stock_ticker['MMA20'] = stock_ticker['Adj Close'].loc[year].rolling(20).mean() #Adj Close 20 MA

# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=stock_ticker["Adj Close"].loc[year], window=20, window_dev=2)

# Bollinger Bands
stock_ticker['BB_Upper'] = indicator_bb.bollinger_hband()
stock_ticker['BB_Lower'] = indicator_bb.bollinger_lband()

# Initialize RSI Indicator
indicator_rsi = RSIIndicator(close=stock_ticker["Adj Close"].loc[year], window=14)

# RSI
stock_ticker['RSI'] = indicator_rsi.rsi()

# Daily Return
stock_ticker['Daily_Return'] = daily_return(stock_ticker["Adj Close"].loc[year])

In [36]:
# Plot the adjusted close price
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':2,'colspan':1}],[None],[{'rowspan':1,'colspan':1}],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker['Adj Close'].loc[year].index,y=stock_ticker['Adj Close'].loc[year],mode="lines",name=f'{ticker}'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Lower'].loc[year].index,y=stock_ticker['BB_Lower'].loc[year],mode="lines",name='BB_Lower'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Upper'].loc[year].index,y=stock_ticker['BB_Upper'].loc[year],mode="lines",name='BB_Upper'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['MMA20'].loc[year].index,y=stock_ticker['MMA20'].loc[year],mode="lines",name='MMA20'),row=1,col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (3,1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year],mode="lines",name='Volume'),row=3, col=1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year].rolling(20).mean(),mode="lines",name='MMA20'),row=3,col=1)
fig.update_yaxes(title_text="Volume", row=3, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker['RSI'].loc[year].index,y=stock_ticker['RSI'].loc[year],mode="lines",name='RSI'),row=4, col=1)
fig.add_hline(y=30, line_width=1, line_dash="dash", line_color="green",row=4,col=1)
fig.add_hline(y=70, line_width=1, line_dash="dash", line_color="red",row=4,col=1)
fig.update_yaxes(title_text="RSI", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker} {year}")

fig.show()

In [37]:
# Plotly
fig = make_subplots()

# Graph (1,1)
# Loop all stock files and get cummulative return for year
for stock in tqdm(index_list):
    f = os.path.join("../input", stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    fig.add_trace(go.Scatter(x=df['Cummulative_Return'].loc[year].index,y=df['Cummulative_Return'].loc[year],mode="lines",name=stock.split('.')[0]),row=1,col=1)

# Update xaxis properties
fig.update_yaxes(title_text="Return", row=1, col=1)

fig.update_layout(height=800, width=1300,showlegend=True,title=f"Cummulative Returns {ticker} for {year}")

fig.show()

  0%|          | 0/51 [00:00<?, ?it/s]

In [40]:
# Create also table of cummulative returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input",stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Cummulative_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Cummulative_Return'])
cum = wiki[['Ticker','Name','Sector']].merge(cum,on='Ticker')
cum.sort_values(by=['Cummulative_Return'],ignore_index=True, ascending=False)


  0%|          | 0/51 [00:00<?, ?it/s]

Ticker                       Name  \
0    TKA.DE            Thyssenkrupp AG   
1    HAG.DE                Hensoldt AG   
2    GBF.DE               Bilfinger SE   
3    KGX.DE              Kion Group AG   
4    LXS.DE                 Lanxess AG   
5    AFX.DE      Carl Zeiss Meditec AG   
6   NDX1.DE                  Nordex SE   
7    KBX.DE            Knorr-Bremse AG   
8   8TRA.DE                  Traton SE   
9   JUN3.DE            Jungheinrich AG   
10   TMV.DE              TeamViewer AG   
11   HOT.DE                Hochtief AG   
12   EVK.DE       Evonik Industries AG   
13   LHA.DE      Deutsche Lufthansa AG   
14   SDF.DE                     K+S AG   
15   BC8.DE                 Bechtle AG   
16  RRTL.DE               RTL Group SA   
17   NEM.DE              Nemetschek SE   
18   EVD.DE  CTS Eventim AG & Co. KGaA   
19  FNTN.DE                 Freenet AG   
20  UTDI.DE         United Internet AG   
21   SAX.DE       Ströer SE & Co. KGaA   
22   NDA.DE                 Aurubis AG   
23  BFSA.DE                Befesa S.A.   
24  FPE3.DE          Fuchs Petrolub SE   
25   FRE.DE  Fresenius Medical Care AG   
26   G1A.DE               GEA Group AG   
27   ^MDAXI                  DAXMidcap   
28   GXI.DE            Gerresheimer AG   
29   KRN.DE                     Krones   
30   G24.DE                 Scout24 AG   
31   WCH.DE           Wacker Chemie AG   
32   TLX.DE                  Talanx AG   
33   STM.DE                STABILUS SE   
34   WAF.DE               Siltronic AG   
35   JEN.DE                Jenoptik AG   
36   RAA.DE                Rational AG   
37  DHER.DE              Delivery Hero   
38   HLE.DE      HELLA GmbH & Co. KGaA   
39   ECV.DE                 Encavis AG   
40  BOSS.DE               Hugo Boss AG   
41   FRA.DE                 Fraport AG   
42   RDC.DE      Redcare Pharmacy N.V.   
43   LEG.DE          LEG Immobilien AG   
44   TEG.DE          TAG Immobilien AG   
45  AIXA.DE                 Aixtron SE   
46  TUI1.DE                     TUI AG   
47   AT1.DE            Aroundtown S.A.   
48   HFG.DE              HelloFresh SE   
49   EVT.DE                  Evotec SE   
50   PUM.DE                    PUMA SE   

                                    Sector       Date  Cummulative_Return  
0                             Conglomerate 2025-03-06          151.564429  
1                                      NaN 2025-03-06          115.848612  
2                                      NaN 2025-03-06           56.304351  
3                       Handling equipment 2025-03-06           45.661089  
4                                Chemistry 2025-03-06           39.601856  
5                       Medical technology 2025-03-06           38.101377  
6                                      NaN 2025-03-06           37.531914  
7                            Manufacturing 2025-03-06           36.043170  
8                                      NaN 2025-03-06           35.815596  
9   Intralogistics, Mechanical engineering 2025-03-06           35.247202  
10                                Software 2025-03-06           33.278314  
11                            Construction 2025-03-06           33.232630  
12                               Chemistry 2025-03-06           32.515701  
13                                Airlines 2025-03-06           30.919134  
14            Mining (fertilizer and salt) 2025-03-06           29.683137  
15                             IT services 2025-03-06           29.674535  
16                                   Media 2025-03-06           26.335174  
17                                Software 2025-03-06           24.841438  
18                          Leisure-events 2025-03-06           24.543857  
19                      Telecommunications 2025-03-06           23.802716  
20                      Telecommunications 2025-03-06           23.352902  
21                                   Media 2025-03-06           22.512975  
22                                  Metals 2025-03-06           22.48677